In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import random

In [2]:
# opening the file in read mode 
my_file = open("../data/names.txt", "r") 
# reading the file 
data = my_file.read()
# split the names into a list
name_list = data.split('\n')
name_list = ["."+x for x in name_list]

In [3]:
chartoix = {char: ix for ix, char in enumerate(sorted(set("".join(name_list))))}
ixtochar = {ix: char for ix, char in enumerate(sorted(set("".join(name_list))))}

In [4]:
def build_x_y(name_list):
    # Create a dataset with, for each example, all possible X and Y combinations we can extract from it
    block_size = 3
    y_list = []
    x_list = []
    for name in name_list:
        for target_ix, target_letter in enumerate(name):
            previous_letters = name[max(-(block_size)+target_ix,0):target_ix] # This will extract the previous 3 letters
            num_pads = block_size - len(previous_letters)
            previous_letters = "."*num_pads + previous_letters
            #print(previous_letters + "-->" + target_letter)
            y_list.append(chartoix[target_letter])
            x_list.append([chartoix[input_letter] for input_letter in previous_letters])
            
    X = torch.tensor(x_list) # Tensor with 3 dimensional list, input letter index
    Y = torch.tensor(y_list) # Tensor with 1 dimensional output
    return X,Y

In [5]:
ix_80_pct = int(0.8*len(name_list))
ix_90_pct = int(0.9*len(name_list))

In [6]:
X_train, Y_train = build_x_y(name_list[:ix_80_pct])
X_val, Y_val= build_x_y(name_list[ix_80_pct:ix_90_pct])
X_test, Y_test = build_x_y(name_list[ix_90_pct:])

In [7]:
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

(torch.Size([182778, 3]),
 torch.Size([182778]),
 torch.Size([22633, 3]),
 torch.Size([22633]),
 torch.Size([22735, 3]),
 torch.Size([22735]))

In [11]:
###### Parameters ########
g = torch.Generator().manual_seed(2147483647)
C = torch.randn(27,10)
W1 = torch.randn(30,300)
#b1 = torch.randn(300) not needed when doing batch norm right after!
# Initializing W2 and b2 with scaled down values ensures we start with a reasonable loss.
W2 = torch.randn(300,27) * 0.1
b2 = torch.randn(27) * 0
bngain = torch.ones((1,300))
bnbias = torch.zeros((1,300))
bnmean_running = torch.zeros((1,300))
bnstd_running = torch.ones((1,300))
parameters = [C, W1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

In [12]:
losses = []
steps = []

for i in range(1000):
    ###### Minibatch #######
    minibatch_size=32
    minibatch_ix = torch.randint(0,len(X_train),(minibatch_size,))
    
    ###### Forward pass ########
    embs = C[X_train[minibatch_ix]]
    h = torch.tanh(embs.view(-1,30) @ W1) #+ b1) #embs.view will reshape. No need to have b1 when doing batch norm.

    # h is calculated to be shape 32,300 (32 examples).
    batch_mean_i = h.mean(0,keepdims=True) # 1,300 mean over all elements in the batch
    batch_std_i = h.std(0,keepdims=True) # 1,300 mean over all elements in the batch
    
    # Keep running tally of the batch norm statistics. This is seperate from the optimization
    with torch.no_grad():
        bnmean_running = 0.999*bnmean_running + 0.001*batch_mean_i
        bnstd_running = 0.999*bnstd_running + 0.001*batch_std_i
        
    # batch norm
    h = bngain*(h-batch_mean_i)/(batch_std_i+1e-5) + bnbias #now, every neuron in its firing rate will be unit gaussian over these 32 examples.


        
    # keep running mean of the values
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,Y_train[minibatch_ix]) #This makes forward/backward pass more efficient and makes things more well behaved numerically for large vals.

    ###### Backward pass ########
    for p in parameters:
        p.grad = None
    loss.backward()
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr*p.grad
    if i % 100 == 0:
        print(loss.item())
    #steps.append(i)
    #losses.append(loss.item())

4.2744598388671875
2.597407579421997
2.447227954864502
2.5438640117645264
2.341860294342041
2.563021183013916
1.8882859945297241
2.348989725112915
2.6667160987854004
2.5552005767822266


In [14]:
# validation loss
embs_val = C[X_val]
h_val = torch.tanh(embs_val.view(-1,30) @ W1)
h_val = bngain*(h_val-bnmean_running)/(bnstd_running) + bnbias #now, every neuron in its firing rate will be unit gaussian over these 32 examples.
logits_val = h_val @ W2 + b2
val_loss = F.cross_entropy(logits_val,Y_val) #This makes forward/backward pass more efficient and makes things more well behaved numerically for large vals.
print(val_loss.item())

2.546466827392578
